# Running Ridge Regression on Garibaldi, train on GTEx, test on MDM

Using scp, you will be able to transfer files between garibaldi and pejlab server

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels
from scipy import stats
import time
import matplotlib.pyplot as plt
import pickle
import random
import csv
import sys
import os

from sklearn.model_selection import KFold
from sklearn.metrics import r2_score

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset, TensorDataset
# Import nn.functional
import torch.nn.functional as F


device = 'cpu'


In [2]:
# trim the gene labels ENSGXXXX.1  -> ENSGXXXX
def trimLabelss(names):
    return [i.split(".")[0] for i in names]

In [ ]:
# split Gene names into different subfiles 100 genes in 1 file  (run 100 models in 1 job)
ind = sys.argv[1]
# Read in genes interested
diagnosedIDs = list(pd.read_csv("/gpfs/home/ydong/data/GTEx/GTExProteinGenes/"+ind+".txt",header = None)[0])

In [ ]:
# prepare output file/folder
if not os.path.exists("/gpfs/home/ydong/data/mini/models/"+ind):
    os.mkdir("/gpfs/home/ydong/data/mini/models/"+ind) 
    
resF = open("/gpfs/home/ydong/data/mini/res/"+ind+".csv", 'a')
header = ["gene","train_R2","val_R2","loss","time"]
resF.write(",".join(header))


### Preprocessing

In [5]:
#import GTEx gene count matrix
normedMat = pd.read_csv("/gpfs/home/ydong/data/logTrimmedNormedCounts.csv",index_col = "Name")

normedMat = normedMat.transpose()

#trim gene labels ENSGXXXX.1  -> ENSGXXXX
normedMat.columns = trimLabelss(normedMat.columns)

In [ ]:
# set the seed for pseudo random
random.seed(123)

# randomly split training and validation set
samples = range(len(normedMat.index))
trainInd = random.sample(samples, 13913)

# Return the unique values in ar1 that are not in ar2.
valInd = np.setdiff1d(samples,trainInd)

In [ ]:
start = time.time()
curr1 = time.time()

# initialize parameters (these turned out to be optimal parameters after testing)
#l2Param is the alpha /normalization coefficient value for l2 normaliztion (Ridge Regression))
l2Param = 1e-5
learningRate = 1e-7
num_epochs = 500


for currID in diagnosedIDs:  

    # grab training data
    x_train, y_train = normedMat.drop(currID,axis = 1).iloc[trainInd,:].to_numpy(), np.reshape(normedMat[currID].iloc[trainInd].to_numpy(), (len(trainInd),1))

    x_val, y_val = normedMat.drop(currID,axis = 1).iloc[valInd,:].to_numpy(), np.reshape(normedMat[currID].iloc[valInd].to_numpy(), (len(valInd),1))

    # create tensor variables
    x_train_tensor = torch.from_numpy(x_train).float().to(device)
    y_train_tensor = torch.from_numpy(y_train).float().to(device)
    x_val_tensor = torch.from_numpy(x_val).float().to(device)
    y_val_tensor = torch.from_numpy(y_val).float().to(device)

    train_data = TensorDataset(x_train_tensor, y_train_tensor)
    val_data = TensorDataset(x_val_tensor,y_val_tensor)

    train_loader = DataLoader(dataset=train_data, batch_size=20, shuffle=True)

    model = nn.Linear(in_features =18848 , out_features = 1)
    #Define loss function
    loss_fn = F.mse_loss
    LOSS = []

    #Define optimizer
    opt = torch.optim.SGD(model.parameters(), lr=learningRate,weight_decay = l2Param)

    # Run the training loop for defined number of epochs
    for epoch in range(num_epochs):

        # Print epoch
        print(f'Starting epoch {epoch+1}')

        # minibatch
        for i, data in enumerate(train_loader, 0):    

            xb,yb = data
            # initialize optimizer
            opt.zero_grad()         

            #Generate predictions
            pred = model(xb)
            loss = loss_fn(pred,yb)

            #Perform gradient descent
            loss.backward()
            opt.step()

    # save current model
    torch.save(model, "/gpfs/home/ydong/data/mini/models/" +ind+"/"+ currID + ".sav")

    
    # record loss of current model    
    lossEpochs = loss.item()

    # R2 of current model
    val_R2 = r2_score(model(x_val_tensor).detach().numpy(),y_val)
    train_R2 = r2_score(model(x_train_tensor).detach().numpy(),y_train)

    currRow = ",".join( [currID,str(train_R2),str(val_R2),str(lossEpochs)])
    resF.write(currRow)
    
    
    curr2 = time.time()
    print(currID + ": " )
    print(curr2 - curr1)
    curr1 = curr2

end = time.time()
print("total time:")
print(end - start)
resF.close()